<a href="https://colab.research.google.com/github/DarthCoder501/ML-AI-Projects/blob/main/Volleyball_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-core langgraph>0.2.27
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.7 MB/s eta 0:00:00


In [2]:
import getpass
import os
import langchain
import json
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langchain.schema import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
langchain_api_key = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key

In [4]:
groq_api_key = userdata.get("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = groq_api_key

In [5]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00


In [7]:
model = ChatGroq(model="llama3-groq-70b-8192-tool-use-preview")

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the prompt template for volleyball training plans
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a professional volleyball coach who specializes in creating personalized training plans. "
            "Gather the player's volleyball position, experience level, and specific volleyball goals. "
            "Generate a comprehensive training plan that focuses exclusively on the player's volleyball goals. "
            "Ensure the plan includes structured phases, specific drills, and clear explanations to facilitate the player's improvement in their desired skill area. "
            "Do not include any fitness-related elements or goals in the training plan, even if mentioned in previous messages. "
            "Do not call external tools or ask follow-up questions if all key information is already provided. "
            "Provide the final training plan directly."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)



In [9]:
class VolleyballState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    position: str
    experience_level: str
    volleyball_goals: Sequence[str]

In [10]:
def generate_volleyball_training_plan(playerPosition, experienceLevel, volleyballGoals):
    return (
        f"Training plan for {playerPosition} with {experienceLevel} experience:\n"
        f"Volleyball Goals: {', '.join(volleyballGoals)}.\n"
        "Include specific drills for each goal to improve your skills."
    )

In [11]:
# Function to call the model and generate training plans
def call_model(state: VolleyballState):
    # Debug: Print state before calling model
    print(f"Calling model with state: {state}")

    # Use the prompt template to call the language model, filling in the user's inputs
    chain = prompt | model
    response = chain.invoke(state)

    # Debug: Print raw response for troubleshooting
    print(f"Model response: {response}")

    # Check if the response contains a tool call
    if "<tool_call>" in response.content:
        # Parse the tool call
        tool_call_str = response.content.strip("<tool_call>").strip("</tool_call>")
        tool_call_data = json.loads(tool_call_str)

        # Call the appropriate function for generating the volleyball training plan
        training_plan = generate_volleyball_training_plan(**tool_call_data['arguments'])
        return {"messages": [training_plan]}

    return {"messages": [response]}

In [12]:
workflow = StateGraph(state_schema=VolleyballState)

# Add nodes and edges
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Create an in-memory checkpoint to save the conversation
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [14]:
# Define the config for conversation thread
config = {"configurable": {"thread_id": "volleyball123"}}

# Start continuous conversation
conversation_history = []

while True:
    # User input
    position = input("What is your volleyball position? (if unsure type 'unsure')): ")
    if position.lower().strip == 'unsure':
        continue
    else:
        position = position.lower().strip()


    experience_level = input("What is your experience level (Beginner, Intermediate, Advanced)? ").lower().strip()

    volleyball_goals_input = input("What are your volleyball goals?: ").lower().strip()

    volleyball_goals = [goal.strip() for goal in volleyball_goals_input.split(",")]

    # Append user's message to the conversation history
    conversation_history.append(
        HumanMessage(
            content=f"I am {position} with {experience_level} experience, "
            f"and my volleyball goals are {volleyball_goals}."
        )
    )

    # Invoke the model with the user's input for personalized training plan
    output = app.invoke(
        {
            "messages": conversation_history,
            "position": position,
            "experience_level": experience_level,
            "volleyball_goals": volleyball_goals
        },
        config
    )

    # Store the response from the model into the conversation history
    conversation_history.append(output["messages"][-1])

    # Print the final output message (training plan)
    if output["messages"]:
        print(output["messages"][-1].content)  # Ensure to print the 'content' of the response
    else:
        print("No response from the model.")
    break

What is your volleyball position? (if unsure type 'unsure')): unsure
What is your experience level (Beginner, Intermediate, Advanced)? beginner
What are your volleyball goals?: be able to do a topsin serve
Calling model with state: {'messages': [HumanMessage(content="I am unsure with beginner experience, and my volleyball goals are ['be able to do a topsin serve'].", additional_kwargs={}, response_metadata={}, id='dcac449a-e3f5-4871-afaf-e2c24d10eb0c')], 'position': 'unsure', 'experience_level': 'beginner', 'volleyball_goals': ['be able to do a topsin serve']}
Model response: content="Training Plan for Beginner Volleyball Player Focusing on Topspin Serve:\n\nPhase 1: Fundamentals of the Topspin Serve (Weeks 1-4)\n\n1. **Stance and Positioning**: Stand with your feet shoulder-width apart, with your body facing the net. Keep your knees slightly bent and your weight evenly distributed between both feet.\n\n2. **Grip**: Hold the ball with your fingertips, ensuring that your thumb is on the